# Diffusion Forward Process Test

이 노트북은 다양한 스케줄러에 대한 forward diffusion process를 테스트하고 비교합니다.

## 🎯 목표
- 다양한 스케줄러 (linear, cosine, step, plateau) 비교
- 단일 이벤트에 대한 노이즈 추가 과정 시각화
- 전체 데이터셋에 대한 통계적 분석
- 스케줄러별 노이즈 패턴 비교


## 1. 환경 설정 및 라이브러리 임포트


In [4]:
import os
import sys
import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path
import subprocess
import warnings
warnings.filterwarnings('ignore')

# Git repository 루트 찾기
def get_git_root():
    try:
        result = subprocess.run(['git', 'rev-parse', '--show-toplevel'], 
                              capture_output=True, text=True, check=True)
        return result.stdout.strip()
    except subprocess.CalledProcessError:
        # Git repository가 아닌 경우 현재 디렉토리 반환
        return str(Path.cwd().parent)

git_root = get_git_root()
project_root = Path(git_root)
sys.path.insert(0, str(project_root))

print(f"🔧 Git repository 루트: {git_root}")
print(f"📁 프로젝트 루트: {project_root}")

# GENESIS 유틸리티 임포트
from config import load_config_from_file, get_default_config
from dataloader.pmt_dataloader import make_dataloader
from models.factory import ModelFactory
from diffusion.gaussian_diffusion import GaussianDiffusion
from utils.denormalization import denormalize_signal, denormalize_label
from utils.event_visualization.event_fast import plot_event_fast
from utils.h5.h5_hist import plot_hist_pair
from utils.event_visualization.event_show import show_event_from_npz
from utils.event_visualization.event_array import show_event_from_array

# 설정
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
print("✅ 라이브러리 임포트 완료!")


🔧 Git repository 루트: /Users/pmj0324/Sicence/IceCube/GENESIS/GENESIS
📁 프로젝트 루트: /Users/pmj0324/Sicence/IceCube/GENESIS/GENESIS
✅ 라이브러리 임포트 완료!


## 2. 스케줄러 설정 로드


In [5]:
# 사용할 스케줄러들
schedulers = {
    'linear': 'configs/training/linear.yaml',
    'cosine': 'configs/training/cosine_annealing.yaml',
    'step': 'configs/training/step.yaml',
    'plateau': 'configs/training/plateau.yaml'
}

configs = {}
diffusion_configs = {}

print("📋 스케줄러 설정 로드 중...")
for name, config_path in schedulers.items():
    try:
        config = load_config_from_file(str(project_root / config_path))
        configs[name] = config
        
        # DiffusionConfig 객체 생성
        from diffusion.gaussian_diffusion import DiffusionConfig
        diffusion_cfg = DiffusionConfig(
            timesteps=config.diffusion.timesteps,
            beta_start=config.diffusion.beta_start,
            beta_end=config.diffusion.beta_end,
            objective=config.diffusion.objective,
            schedule=config.diffusion.schedule,
            use_cfg=getattr(config.diffusion, 'use_cfg', True),
            cfg_scale=getattr(config.diffusion, 'cfg_scale', 2.0),
            cfg_dropout=getattr(config.diffusion, 'cfg_dropout', 0.1)
        )
        diffusion_configs[name] = diffusion_cfg
        
        print(f"✅ {name.upper()} 스케줄러 로드 완료 - timesteps: {config.diffusion.timesteps}")
        
    except Exception as e:
        print(f"❌ {name.upper()} 스케줄러 로드 실패: {e}")

print(f"\n🎯 총 {len(configs)}개의 스케줄러 설정 완료!")


📋 스케줄러 설정 로드 중...
❌ LINEAR 스케줄러 로드 실패: ModelConfig.__init__() got an unexpected keyword argument 'exclude_zero_time'
✅ COSINE 스케줄러 로드 완료 - timesteps: 1000
❌ STEP 스케줄러 로드 실패: ModelConfig.__init__() got an unexpected keyword argument 'exclude_zero_time'
❌ PLATEAU 스케줄러 로드 실패: ModelConfig.__init__() got an unexpected keyword argument 'exclude_zero_time'

🎯 총 1개의 스케줄러 설정 완료!


## 3. 데이터 로더 생성 및 단일 이벤트 Forward Diffusion 비교


In [6]:
# 첫 번째 설정을 사용하여 데이터 로더 생성
main_config = configs[list(configs.keys())[0]]
print(f"📊 메인 설정 사용: {list(configs.keys())[0].upper()}")

# H5 파일 경로 수정 (절대 경로 사용)
h5_path = str(project_root / main_config.data.h5_path)
print(f"📁 H5 파일 경로: {h5_path}")

# 데이터 로더 생성
dataloader = make_dataloader(
    h5_path,
    batch_size=4,  # 테스트용으로 작은 배치
    shuffle=True,
    num_workers=0,
    replace_time_inf_with=main_config.data.replace_time_inf_with,
    channel_first=main_config.data.channel_first
)

# 하나의 배치 가져오기
x_sig, geom, label, idx = next(iter(dataloader))
print(f"✅ 데이터 로드 완료!")
print(f"📐 신호 shape: {x_sig.shape}")
print(f"📐 기하학 shape: {geom.shape}")
print(f"📐 라벨 shape: {label.shape}")

# 테스트할 timestep들
test_timesteps = [0, 100, 250, 500, 750, 999]
event_idx = 0  # 첫 번째 이벤트 사용

# 원본 이벤트
original_signal = x_sig[event_idx].clone()
original_geom = geom[event_idx].clone()
original_label = label[event_idx].clone()

print(f"\n🎯 이벤트 {event_idx} Forward Diffusion 테스트")
print(f"📊 테스트 timesteps: {test_timesteps}")

# 각 스케줄러별 결과 저장
scheduler_results = {}

# 더미 모델 생성 (forward diffusion만 테스트하기 위해)
class DummyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x, t, label=None, geom=None):
        return x  # 실제로는 사용하지 않음

dummy_model = DummyModel()

for scheduler_name, diffusion_cfg in diffusion_configs.items():
    print(f"\n🔄 {scheduler_name.upper()} 스케줄러 처리 중...")
    
    # GaussianDiffusion 객체 생성
    diffusion = GaussianDiffusion(dummy_model, diffusion_cfg)
    
    results = {}
    for t in test_timesteps:
        # Forward diffusion 적용
        noisy_signal = diffusion.forward_diffusion(original_signal.unsqueeze(0), t)
        results[t] = noisy_signal.squeeze(0)
        
        print(f"  ✅ Timestep {t} 완료")
    
    scheduler_results[scheduler_name] = results
    print(f"✅ {scheduler_name.upper()} 완료!")

print(f"\n🎉 모든 스케줄러 Forward Diffusion 완료!")


📊 메인 설정 사용: COSINE
📁 H5 파일 경로: /Users/pmj0324/Sicence/IceCube/GENESIS/GENESIS/GENESIS-data/22644_0921_time_shift.h5
✅ 데이터 로드 완료!
📐 신호 shape: torch.Size([4, 2, 5160])
📐 기하학 shape: torch.Size([4, 3, 5160])
📐 라벨 shape: torch.Size([4, 6])

🎯 이벤트 0 Forward Diffusion 테스트
📊 테스트 timesteps: [0, 100, 250, 500, 750, 999]

🔄 COSINE 스케줄러 처리 중...


AttributeError: 'GaussianDiffusion' object has no attribute 'forward_diffusion'

## 4. 스케줄러별 노이즈 패턴 시각화


In [ ]:
# 스케줄러별 노이즈 패턴 비교
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
axes = axes.flatten()

colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']

for i, (scheduler_name, results) in enumerate(scheduler_results.items()):
    ax = axes[i]
    
    # Charge 채널만 시각화 (첫 번째 채널)
    for j, (t, noisy_signal) in enumerate(results.items()):
        charge_data = noisy_signal[0].cpu().numpy()  # Charge 채널
        
        # 히스토그램
        ax.hist(charge_data, bins=50, alpha=0.6, 
               label=f't={t}', color=colors[j % len(colors)], density=True)
    
    ax.set_title(f'{scheduler_name.upper()} Scheduler - Charge Distribution')
    ax.set_xlabel('Charge Value')
    ax.set_ylabel('Density')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(str(project_root / 'testing' / 'scheduler_comparison_charge.png'), 
            dpi=150, bbox_inches='tight')
plt.show()

print("✅ 스케줄러별 Charge 분포 비교 완료!")


## 5. Time 채널 비교 및 통계 분석


In [ ]:
# Time 채널 비교
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
axes = axes.flatten()

for i, (scheduler_name, results) in enumerate(scheduler_results.items()):
    ax = axes[i]
    
    # Time 채널 시각화 (두 번째 채널)
    for j, (t, noisy_signal) in enumerate(results.items()):
        time_data = noisy_signal[1].cpu().numpy()  # Time 채널
        
        # 0이 아닌 값들만 히스토그램
        non_zero_time = time_data[time_data > 0]
        if len(non_zero_time) > 0:
            ax.hist(non_zero_time, bins=50, alpha=0.6, 
                   label=f't={t}', color=colors[j % len(colors)], density=True)
    
    ax.set_title(f'{scheduler_name.upper()} Scheduler - Time Distribution (Non-zero)')
    ax.set_xlabel('Time Value')
    ax.set_ylabel('Density')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')  # 로그 스케일

plt.tight_layout()
plt.savefig(str(project_root / 'testing' / 'scheduler_comparison_time.png'), 
            dpi=150, bbox_inches='tight')
plt.show()

print("✅ 스케줄러별 Time 분포 비교 완료!")

# 각 스케줄러별 통계 계산
stats_results = {}

for scheduler_name, results in scheduler_results.items():
    stats = {}
    
    for t, noisy_signal in results.items():
        charge_data = noisy_signal[0].cpu().numpy()
        time_data = noisy_signal[1].cpu().numpy()
        
        stats[t] = {
            'charge': {
                'mean': np.mean(charge_data),
                'std': np.std(charge_data),
                'min': np.min(charge_data),
                'max': np.max(charge_data)
            },
            'time': {
                'mean': np.mean(time_data[time_data > 0]),
                'std': np.std(time_data[time_data > 0]),
                'min': np.min(time_data[time_data > 0]),
                'max': np.max(time_data[time_data > 0])
            }
        }
    
    stats_results[scheduler_name] = stats

# 통계 결과 시각화
fig, axes = plt.subplots(2, 2, figsize=(20, 15))

# Charge Mean
ax = axes[0, 0]
for scheduler_name, stats in stats_results.items():
    timesteps = list(stats.keys())
    means = [stats[t]['charge']['mean'] for t in timesteps]
    ax.plot(timesteps, means, 'o-', label=scheduler_name.upper(), linewidth=2)
ax.set_title('Charge Mean vs Timestep')
ax.set_xlabel('Timestep')
ax.set_ylabel('Mean Charge')
ax.legend()
ax.grid(True, alpha=0.3)

# Charge Std
ax = axes[0, 1]
for scheduler_name, stats in stats_results.items():
    timesteps = list(stats.keys())
    stds = [stats[t]['charge']['std'] for t in timesteps]
    ax.plot(timesteps, stds, 'o-', label=scheduler_name.upper(), linewidth=2)
ax.set_title('Charge Std vs Timestep')
ax.set_xlabel('Timestep')
ax.set_ylabel('Std Charge')
ax.legend()
ax.grid(True, alpha=0.3)

# Time Mean
ax = axes[1, 0]
for scheduler_name, stats in stats_results.items():
    timesteps = list(stats.keys())
    means = [stats[t]['time']['mean'] for t in timesteps]
    ax.plot(timesteps, means, 'o-', label=scheduler_name.upper(), linewidth=2)
ax.set_title('Time Mean vs Timestep (Non-zero)')
ax.set_xlabel('Timestep')
ax.set_ylabel('Mean Time')
ax.legend()
ax.grid(True, alpha=0.3)

# Time Std
ax = axes[1, 1]
for scheduler_name, stats in stats_results.items():
    timesteps = list(stats.keys())
    stds = [stats[t]['time']['std'] for t in timesteps]
    ax.plot(timesteps, stds, 'o-', label=scheduler_name.upper(), linewidth=2)
ax.set_title('Time Std vs Timestep (Non-zero)')
ax.set_xlabel('Timestep')
ax.set_ylabel('Std Time')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(str(project_root / 'testing' / 'scheduler_statistics.png'), 
            dpi=150, bbox_inches='tight')
plt.show()

print("✅ 스케줄러별 통계 분석 완료!")


## 6. 전체 데이터셋 통계 분석


In [ ]:
# 전체 데이터셋에 대한 통계적 분석
print("📊 전체 데이터셋 통계 분석 시작...")

# 더 큰 배치로 분석
batch_size = 32
large_dataloader = make_dataloader(
    h5_path,  # 절대 경로 사용
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    replace_time_inf_with=main_config.data.replace_time_inf_with,
    channel_first=main_config.data.channel_first
)

# 여러 배치 수집
all_signals = []
num_batches = 10  # 10개 배치 분석

for i, (x_sig_batch, _, _, _) in enumerate(large_dataloader):
    if i >= num_batches:
        break
    all_signals.append(x_sig_batch)
    print(f"  📦 배치 {i+1}/{num_batches} 수집 완료")

# 모든 신호 결합
all_signals = torch.cat(all_signals, dim=0)
print(f"✅ 총 {all_signals.shape[0]}개 이벤트 수집 완료!")

# 각 스케줄러별로 전체 데이터셋 분석
dataset_stats = {}
target_timestep = 500

for scheduler_name, diffusion_cfg in diffusion_configs.items():
    print(f"\n🔄 {scheduler_name.upper()} 스케줄러로 전체 데이터셋 분석 중...")
    
    # GaussianDiffusion 객체 생성
    diffusion = GaussianDiffusion(dummy_model, diffusion_cfg)
    
    # 배치별로 forward diffusion 적용
    noisy_signals = []
    batch_size_analysis = 8  # 분석용 배치 크기
    
    for i in range(0, all_signals.shape[0], batch_size_analysis):
        batch = all_signals[i:i+batch_size_analysis]
        noisy_batch = diffusion.forward_diffusion(batch, target_timestep)
        noisy_signals.append(noisy_batch)
        
        if (i // batch_size_analysis + 1) % 5 == 0:
            print(f"  📊 {i // batch_size_analysis + 1}배치 완료")
    
    # 모든 배치 결합
    all_noisy = torch.cat(noisy_signals, dim=0)
    
    # 통계 계산
    charge_stats = {
        'mean': torch.mean(all_noisy[:, 0]).item(),
        'std': torch.std(all_noisy[:, 0]).item(),
        'min': torch.min(all_noisy[:, 0]).item(),
        'max': torch.max(all_noisy[:, 0]).item()
    }
    
    # Time 통계 (0이 아닌 값들만)
    time_data = all_noisy[:, 1]
    non_zero_mask = time_data > 0
    if non_zero_mask.sum() > 0:
        time_nonzero = time_data[non_zero_mask]
        time_stats = {
            'mean': torch.mean(time_nonzero).item(),
            'std': torch.std(time_nonzero).item(),
            'min': torch.min(time_nonzero).item(),
            'max': torch.max(time_nonzero).item(),
            'non_zero_ratio': non_zero_mask.float().mean().item()
        }
    else:
        time_stats = {'mean': 0, 'std': 0, 'min': 0, 'max': 0, 'non_zero_ratio': 0}
    
    dataset_stats[scheduler_name] = {
        'charge': charge_stats,
        'time': time_stats,
        'num_events': all_noisy.shape[0]
    }
    
    print(f"✅ {scheduler_name.upper()} 분석 완료 - {all_noisy.shape[0]}개 이벤트")

print("\n🎉 전체 데이터셋 분석 완료!")


## 7. 결과 요약 및 비교


In [ ]:
# 결과 요약 테이블
print("\n" + "="*80)
print("📊 DIFFUSION SCHEDULER COMPARISON RESULTS")
print("="*80)

print(f"\n🎯 분석 대상: {target_timestep} timestep에서 {dataset_stats[list(dataset_stats.keys())[0]]['num_events']}개 이벤트")
print("\n📋 스케줄러별 통계 비교:")
print("-"*80)

for scheduler_name, stats in dataset_stats.items():
    print(f"\n🔸 {scheduler_name.upper()} SCHEDULER:")
    print(f"  📊 Charge - Mean: {stats['charge']['mean']:.4f}, Std: {stats['charge']['std']:.4f}")
    print(f"      Range: [{stats['charge']['min']:.4f}, {stats['charge']['max']:.4f}]")
    print(f"  ⏰ Time   - Mean: {stats['time']['mean']:.4f}, Std: {stats['time']['std']:.4f}")
    print(f"      Range: [{stats['time']['min']:.4f}, {stats['time']['max']:.4f}]")
    print(f"      Non-zero ratio: {stats['time']['non_zero_ratio']:.3f}")

# 비교 차트
fig, axes = plt.subplots(2, 2, figsize=(20, 15))

scheduler_names = list(dataset_stats.keys())
charge_means = [dataset_stats[name]['charge']['mean'] for name in scheduler_names]
charge_stds = [dataset_stats[name]['charge']['std'] for name in scheduler_names]
time_means = [dataset_stats[name]['time']['mean'] for name in scheduler_names]
time_stds = [dataset_stats[name]['time']['std'] for name in scheduler_names]

# Charge Mean 비교
axes[0, 0].bar(scheduler_names, charge_means, color=['red', 'blue', 'green', 'orange'])
axes[0, 0].set_title('Charge Mean by Scheduler')
axes[0, 0].set_ylabel('Mean Charge')
axes[0, 0].tick_params(axis='x', rotation=45)

# Charge Std 비교
axes[0, 1].bar(scheduler_names, charge_stds, color=['red', 'blue', 'green', 'orange'])
axes[0, 1].set_title('Charge Std by Scheduler')
axes[0, 1].set_ylabel('Std Charge')
axes[0, 1].tick_params(axis='x', rotation=45)

# Time Mean 비교
axes[1, 0].bar(scheduler_names, time_means, color=['red', 'blue', 'green', 'orange'])
axes[1, 0].set_title('Time Mean by Scheduler (Non-zero)')
axes[1, 0].set_ylabel('Mean Time')
axes[1, 0].tick_params(axis='x', rotation=45)

# Time Std 비교
axes[1, 1].bar(scheduler_names, time_stds, color=['red', 'blue', 'green', 'orange'])
axes[1, 1].set_title('Time Std by Scheduler (Non-zero)')
axes[1, 1].set_ylabel('Std Time')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(str(project_root / 'testing' / 'scheduler_final_comparison.png'), 
            dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "="*80)
print("✅ 모든 분석 완료! 결과 이미지들이 testing/ 폴더에 저장되었습니다.")
print("📁 저장된 파일들:")
print("  - scheduler_comparison_charge.png")
print("  - scheduler_comparison_time.png")
print("  - scheduler_statistics.png")
print("  - scheduler_3d_comparison.png")
print("  - scheduler_final_comparison.png")
print("="*80)
